# Introduction

Encoding / decoding pipeline using Huffman Code.

Erik Martin Welin, 2025-05

In [168]:
# Definition of binary tree node for this problem.

class Node:
    def __init__(self, freq, symbol=None, left=None, right=None):
        self.freq = freq
        self.symbol = symbol
        self.left = left
        self.right = right

# lt lets the heap know how to compare nodes for the purposes of priority.

    def __lt__(self, other):
        return self.freq < other.freq

In [169]:
# Fetch document to encode.

# This block is optional. You can comment it out and set your own document path

import os

root = '/content' # /content is the root dir in Google Colab.

document = os.path.join(root, 'text/Illiad_song1_Planet_Ebook.docx')

assert os.path.exists(document), 'make sure the document path describes a valid file.docx'

In [170]:
! pip install docx2txt

In [171]:
import docx2txt

s = docx2txt.process(document)

In [172]:
from collections import Counter

freqs = Counter(s)

total = sum(freqs.values())
source = {}
for key, val in freqs.items():
  source[key] = val / total

In [173]:
# Implementation of the Huffman algorithm using a prio q and a binary tree.

import heapq


def Huffman_algorithm(source):

  # create leaf nodes for the source and add them to the heap.

  heap = []

  for key, val in source.items():
    leaf = Node(freq = val, symbol = key)
    heapq.heappush(heap, leaf)


  while len(heap) >1 :
    left = heapq.heappop(heap)
    right = heapq.heappop(heap)

    # create parent node with combined probabilities of the previous and
    # push the new node to the heap

    new_node = Node(freq = left.freq + right.freq, left = left, right = right)
    heapq.heappush(heap, new_node)

  # at this point only the root remains.

  root = heapq.heappop(heap)
  assert len(heap) == 0

  return root

root = Huffman_algorithm(source)

In [174]:
# build codewords by traversing the binary tree.
# Assign 0 for a left path and 1 for a right path.

def build_codewords(node, map_dict, path_so_far = "" ):

    # if node is a leaf:
    #     save mapping: symbol --> codeword

    if node.left is None and node.right is None:
        map_dict[node.symbol] = path_so_far

    # else:
    #     continue traversing.

    else:
        build_codewords(node.left, map_dict, path_so_far + '0')
        build_codewords(node.right,  map_dict, path_so_far + '1')

map_dict = {}
build_codewords(root, map_dict)

# Compute the average code-length and compare it to the Shannon entropy.



In [175]:
"""
the theoretical bounds on the average code length L is
H <= L <= H +1 bits
Where H is the Shannon entropy of the source.
"""

'\nthe theoretical bounds on the average code length L is\nH <= L <= H +1 bits\nWhere H is the Shannon entropy of the source.\n'

In [176]:
import math

"""
this function:
given a list of estimated probabilities, calculate Shannon Entropy in bits.

0 log(0) is mathematically undefined but:
in the limit as x in xlog(x) goes to 0+ the expression evaluates to 0.
As a consequence, only p(x) for x > 0 contributes to the entropy.
"""


def entropy(lst):
  return -sum(x * math.log2(x) for x in lst if x > 0)


In [177]:
# Compute average code length L and compare it to the entropy H

L = 0

for symbol, probability in source.items():
  L += probability * len( map_dict[symbol] )

probabilities = list(source.values())
H = entropy(probabilities)
error = abs(H - L)

print(f"{'Average codword length:':<30}{L:>8.3f} bits")
print('\n')
print(f"Theoretical bounds on the average code length given by the entropy:\n")
print(f"{'Lower:':<30}{H:>8.3f} bits")
print(f"{'Upper:':<30}{(H + 1):>8.3f} bits\n")
print(f"{'Absolute error:':<30}{error:>8.3f} bits")

Average codword length:          4.365 bits


Theoretical bounds on the average code length given by the entropy:

Lower:                           4.328 bits
Upper:                           5.328 bits

Absolute error:                  0.037 bits


# Go from bitstream to bytes.

Create a list of bytes from my bitstring. Pad the length of the last byte to 8 (one byte).


In [178]:
bitstring = ''

for symbol in s:
  bitstring = bitstring + (map_dict[symbol])

# 8-len(s)%8 adds an additional byte when len(s)%8 == 0.
# taking modulo 8 again avoids this inconvinience.

padding = (8-len(bitstring)%8) % 8
bitstring_padded = bitstring + "0" * padding

assert len(bitstring_padded)%8 == 0

# convert the bitstring to bytes and store in a list.
# int(s,2) converts the binary string, s, to integer.

byte_list = [
    int(bitstring_padded[i:i+8], 2)
    for i in range(0, len(bitstring_padded), 8)
]
byte_list[:5]

[129, 54, 145, 70, 66]

# Decoding

Decode the encoded byte-stream. This decoding scheme works if and only if map_dict describes a prefix free code, such as the Huffman code.

In [179]:
r_bitstring = ''

r_bitstring = ''.join(f'{byte:08b}' for byte in byte_list)

# remove the padded bits

r_bitstring = r_bitstring[:-padding]

In [180]:
# build dict reverse_mapping:
# reverse_mapping[bitstring] = symbol

reverse_mapping = {value: key for key,value in map_dict.items()}

# build codewords from the bitstream.

decoded_string = ''
temp_codeword = ''

for bit in r_bitstring:
  temp_codeword = temp_codeword + bit
  if temp_codeword in reverse_mapping:
    decoded_string += reverse_mapping[temp_codeword]
    temp_codeword = ''

In [181]:
print(decoded_string[:3000])

Sing, O goddess, the anger of Achilles son of Peleus, that  

brought countless ills upon the Achaeans. Many a brave  soul did it send hurrying down to Hades, and many a hero  did it yield a prey to dogs and vultures, for so were the  counsels of Jove fulfilled from the day on which the son of  Atreus, king of men, and great Achilles, first fell out with  one another. 

And which of the gods was it that set them on to quar rel? It was the son of Jove and Leto; for he was angry with  the king and sent a pestilence upon the host to plague the  people, because the son of Atreus had dishonoured Chry ses his priest. Now Chryses had come to the ships of the  Achaeans to free his daughter, and had brought with him a  great ransom: moreover he bore in his hand the sceptre of  Apollo wreathed with a suppliant’s wreath, and he besought  the Achaeans, but most of all the two sons of Atreus, who  were their chiefs. 

‘Sons of Atreus,’ he cried, ‘and all other Achaeans, may  the gods who dwell in O